## IMPORT LIBRARIES

In [8]:
# Import necessary libraries
import tkinter as tk
from tkinter import simpledialog, filedialog
import pandas as pd
from pyDOE2 import pbdesign
import os


## CREATE FACTORIAL DESIGN PLAN

In [9]:
# Function to generate a DataFrame with factor names
def generate_factors_df(factors):
    """
    Generate a DataFrame with factor names.

    Args:
    - factors (list): List of factor names.

    Returns:
    - df (DataFrame): DataFrame representing factor names.
    """
    data = {'Factor': factors}
    df = pd.DataFrame(data)
    return df


# Function to generate a Plackett-Burman design matrix based on given factors
def create_plackett_burman_design(factors_df):
    """
    Generates a Plackett-Burman design matrix based on the given factors.

    Args:
    - factors_df (DataFrame): DataFrame containing the factor names and levels.

    Returns:
    - df (DataFrame): DataFrame containing the design matrix.
    """
    factors = factors_df['Factor'].tolist()
    
    # Generate Plackett-Burman design
    design = pbdesign(len(factors))

    design = -design

    # Convert design matrix back to original factors
    df = pd.DataFrame(design, columns=factors)
    
    return df

# Define the custom mapping function
def custom_mapping(value, num_levels):
    """
    Custom mapping function to transform values according to specific rules.

    Args:
    - value (int): Value representing factor level.
    - num_levels (int): Number of levels for the factor.

    Returns:
    - mapped_value (int): Transformed value.
    """
    # Map values according to specific rules
    if value == 1:
        return 0 if 2 in range(num_levels) else 1
    elif value == 2:
        return 1
    elif value == 0:
        return -1
    else:
        return value
    
def get_user_input():
    """
    Prompt the user to input factors, number of replicates, Excel file name, and export folder.

    Returns:
    - factors (list): List of factor names.
    - num_replicates (int): Number of replicates.
    - excel_file_name (str): Excel file name.
    - export_folder (str): Export folder path.
    """
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Prompt user for the number of factors
    num_factors = simpledialog.askinteger("Input", "Enter the number of factors:")

    factors = []

    # Prompt user to enter factor names
    for i in range(num_factors):
        factor = simpledialog.askstring("Input", f"Enter the name of factor {i+1}:")
        factors.append(factor)

    # Prompt user to enter the number of replicates
    num_replicates = simpledialog.askinteger("Input", "Enter the number of replicates:")

    # Prompt user to enter the Excel file name
    excel_file_name = simpledialog.askstring("Input", "Enter the Excel file name:")

    # Prompt user to select the export folder
    export_folder = filedialog.askdirectory(title="Select Export Folder")

    return factors, num_replicates, excel_file_name, export_folder

# Main function
def main():
    try:
        # Get user input
        factors, num_replicates, excel_file_name, export_folder = get_user_input()

        # Generate factor levels DataFrame
        factor_levels_df = generate_factors_df(factors)

        # Create full factorial design DataFrame
        full_factorial_df = create_plackett_burman_design(factor_levels_df)

        # Apply custom mapping to each value in the DataFrame
        for column in full_factorial_df.columns:
            num_levels = 2  # As levels are always 2
            full_factorial_df[column] = full_factorial_df[column].apply(lambda x: custom_mapping(x, num_levels))

        # Duplicate the full factorial design DataFrame
        full_factorial_df_duplicated = pd.concat([full_factorial_df] * num_replicates, ignore_index=True)

        # Merge the two DataFrames
        merged_df = pd.concat([factor_levels_df, pd.DataFrame(columns=['']), full_factorial_df_duplicated], axis=1)

        # Add a blank column with heading 'Results' after the last factor
        merged_df.insert(len(merged_df.columns), 'Results', '')

        # Append '.xlsx' extension if not provided
        if not excel_file_name.endswith('.xlsx'):
            excel_file_name += '.xlsx'

        # Write the merged DataFrame to an Excel file in the export folder
        excel_file_path = os.path.join(export_folder, excel_file_name)
        merged_df.to_excel(excel_file_path, index=False)
        print("Excel file saved successfully.")

    except Exception as e:
        print("An error occurred:", e)

if __name__ == "__main__":
    main()


Excel file saved successfully.
